<h1><center>Tarea 2 - Análisis de sentimiento

<center>Kin Nevárez<center>
    
<center>26 de mayo 2022<center>

<center>Facultad de Ciencias Físico Matemáticas<center>

<center>Maestría en Ciencia de Datos<center>

___

# 1. Resumen

El código a continuación es una ejemplificación de los tipos de pre procesamiento que se pueden realizar para un correcto y oportuno análisis de texto, el cual, se puede observar en gráficas de frecuencia y estadísticas de palabras más usadas.

Posteriormente, se realiza un análisis de sentimiento con tres módulos distintos y se comparan los resultados de dicho análisis.

La base de datos utilizada tiene el nombre "First GOP Debate Twitter Sentiment" y fue obtenida del repositorio de Kaggle, en la siguiente liga: https://www.kaggle.com/datasets/crowdflower/first-gop-debate-twitter-sentiment?resource=download

Esta base de datos contiene más de 10,000 tweets sobre el primer debate presidencial de 2016 celebrado en Ohio.

______________

# 2. Establecimiento del espacio de trabajo

Se importan las librerías necesarias para las funciones utilizadas.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Para trabajar con data.frames
import pandas as pd

#Para limpieza de texto
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from nltk.stem import WordNetLemmatizer

In [3]:
#Para análisis de sentimiento con sentiwordnet
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn

#Para análisis de sentimiento con textblob
from textblob import TextBlob

#Para análisis de sentimiento con vader
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#visualización de datos
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

___

# 3. Acceso a los datos

Se lee el csv descargado:

In [5]:
import os
os.chdir('/content/drive/MyDrive/MAESTRÍA/TETRAMESTRE 4/PROCESAMIENTO Y CLASIFICACIÓN/TAREA 2')

In [6]:
tweets=pd.read_csv('Sentiment.csv',encoding='utf-8')
tweets.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


___

# 4. Pre - procesamiento

### 4.1 Limpieza

Al estar trabajando con tweets, es necesario hacer limpieza de datos muy específica, la cual contempla los siguientes pasos:

In [45]:
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    # Unir de nuevo las palabras
    texto_final = ' '.join([str(item) for item in nuevo_texto])
    return(texto_final)

In [46]:
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto crudo:')
print(tweets['text'][0])
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio:')
print(limpiar_tokenizar(texto=tweets['text'][0]))

-----------------------------------------------------------------------------------------------------------------
Texto crudo:
RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate
-----------------------------------------------------------------------------------------------------------------
Texto limpio:
rt nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate


In [47]:
tweets['clean_text'] = tweets['text'].apply(lambda x: limpiar_tokenizar(x))
tweets[['name','text', 'clean_text']].head().head()

,name,text,clean_text
0,I_Am_Kenzi,RT @NancyLeeGrahn: How did everyone feel about...,rt nancyleegrahn how did everyone feel about t...
1,PeacefulQuest,RT @ScottWalker: Didn't catch the full #GOPdeb...,rt scottwalker didn catch the full gopdebate l...
2,PussssyCroook,RT @TJMShow: No mention of Tamir Rice and the ...,rt tjmshow no mention of tamir rice and the go...
3,MattFromTexas31,RT @RobGeorge: That Carly Fiorina is trending ...,rt robgeorge that carly fiorina is trending ho...
4,sharonDay5,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,rt danscavino gopdebate realdonaldtrump delive...


Se puede observar que fueron eliminados los caracteres especiales, sobre todo los más frecuentes como el "@" y el "#" utilizados para mencionar usuarios y crear tags respectivamente.

### 4.2 Stop Words

Se eliminan las palabras más frecuentes en el idioma inglés. Éstas no proporcionan información relevante en los textos analizados.

A diferencia de otras ocasiones, se usa módulo gensim pues contiene un listado de stopwords más completo que el comúnmente utilizado en nltk. Para más información sobre este módulo,visitar: https://stackabuse.com/removing-stop-words-from-strings-in-python/.

Aunado a esto, se agregan palabras que en el contexto actual no son relevantes, como "rt" que indica un retweet y está presente en muchos textos, asícomo "gobdebate" que está incluido como etiqueta #GOBDebate.

In [48]:
my_sw = STOPWORDS.union(set(['rt','gobdebate','gopdebate','gopdebates']))

In [49]:
def remove_sw(text): 
    #Tokenización
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in my_sw]
    # Unir de nuevo las palabras
    texto_final = ' '.join([str(item) for item in tokens_without_sw])
    return(texto_final)

In [50]:
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio con stopwords:')
print(tweets['clean_text'][0])
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio sin stopwords:')
print(remove_sw(tweets['clean_text'][0]))

-----------------------------------------------------------------------------------------------------------------
Texto limpio con stopwords:
rt nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate
-----------------------------------------------------------------------------------------------------------------
Texto limpio sin stopwords:
nancyleegrahn feel climate change question night exactly


In [51]:
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: remove_sw(x))

In [52]:
tweets[['text','clean_text']].head(10)

,text,clean_text
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question nig...
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best lines secon...
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice held cleveland wow
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hours debate ...
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump delivered highest r...
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal exec...
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman liked happy heard going moderator...
7,Going on #MSNBC Live with @ThomasARoberts arou...,going msnbc live thomasaroberts pm et
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlights lizzwinstead ben carson brain ...
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate proved batsask badas...


### 4.3 Origen de palabras y lematización

Se hace uso de un diccionario que contiene el tipo de palabra utilizada en la oración: adjetivo, verbo, sustantivo y adverbio para poder obtener la palabra raíz de ésta, por medio de la lematización.

In [53]:
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

In [54]:
def pos_tag_text(text):
    tags = pos_tag(word_tokenize(text))
    tagged = []
    for word, tag in tags:
        tagged.append(tuple([word, pos_dict.get(tag[0])]))
    return tagged

In [55]:
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio sin stopwords, sin tag:')
print(tweets['clean_text'][0])
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio sin stopwords, con tag:')
print(pos_tag_text(tweets['clean_text'][0]))

-----------------------------------------------------------------------------------------------------------------
Texto limpio sin stopwords, sin tag:
nancyleegrahn feel climate change question night exactly
-----------------------------------------------------------------------------------------------------------------
Texto limpio sin stopwords, con tag:
[('nancyleegrahn', 'a'), ('feel', 'n'), ('climate', 'n'), ('change', 'n'), ('question', 'n'), ('night', 'n'), ('exactly', 'r')]


In [56]:
tweets['pos_tagged'] = tweets['clean_text'].apply(pos_tag_text)

In [57]:
tweets[['text','clean_text','pos_tagged']].head(10)

,text,clean_text,pos_tagged
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question nig...,"[(nancyleegrahn, a), (feel, n), (climate, n), ..."
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best lines secon...,"[(scottwalker, n), (catch, n), (night, n), (sc..."
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice held cleveland wow,"[(tjmshow, n), (mention, n), (tamir, n), (rice..."
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hours debate ...,"[(robgeorge, n), (carly, a), (fiorina, n), (tr..."
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump delivered highest r...,"[(danscavino, n), (realdonaldtrump, n), (deliv..."
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal exec...,"[(gregabbott, n), (tx, n), (tedcruz, a), (day,..."
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman liked happy heard going moderator...,"[(warriorwoman, n), (liked, v), (happy, a), (h..."
7,Going on #MSNBC Live with @ThomasARoberts arou...,going msnbc live thomasaroberts pm et,"[(going, v), (msnbc, a), (live, a), (thomasaro..."
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlights lizzwinstead ben carson brain ...,"[(deer, n), (headlights, n), (lizzwinstead, v)..."
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate proved batsask badas...,"[(nancyosborne, a), (night, n), (debate, n), (..."


In [58]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos: 
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:  
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew
   

In [59]:
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio sin stopwords, con tag, sin lematización:')
print(tweets['pos_tagged'][7])
print('-----------------------------------------------------------------------------------------------------------------')
print('Texto limpio sin stopwords, con tag, con lematización:')
print(lemmatize(tweets['pos_tagged'][7]))

-----------------------------------------------------------------------------------------------------------------
Texto limpio sin stopwords, con tag, sin lematización:
[('going', 'v'), ('msnbc', 'a'), ('live', 'a'), ('thomasaroberts', 'n'), ('pm', 'v'), ('et', 'n')]
-----------------------------------------------------------------------------------------------------------------
Texto limpio sin stopwords, con tag, con lematización:
  go msnbc live thomasaroberts pm et


In [60]:
tweets['lemma_text'] = tweets['pos_tagged'].apply(lemmatize)

In [142]:
tweets[['text','lemma_text']].head(10)

,text,lemma_text
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question n...
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best line seco...
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice hold cleveland wow
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hour debate...
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump deliver high rati...
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal ex...
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman like happy heard go moderator a...
7,Going on #MSNBC Live with @ThomasARoberts arou...,go msnbc live thomasaroberts pm et
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlight lizzwinstead ben carson brain...
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate prove batsask bada...


Al final lo que se observa es el texto limpio sin caracteres especiales, sin términos irrelevantes y con la palabra raíz, lo cual hará mucho más sencillo el análisis de sentimientos posterior.

_______

# 5. Análisis de Sentimiento


### 5.1 Con librería SentiWordNet

Para poder usar las librerías, es necesario conocer cómo funciona el análisis de sentimiento. De acuerdo con SentiWordNet, cada término tiene asociados una puntuación positiva, negativa y neutral:

$$ Pos(t)+ Neg(t) + Neu(t) = 1 $$

De esta forma, el score final, se encuentra en un rango entre $[0,1]$, y se asigna el sentimiento de la puntuación más alta.

In [125]:
def sentiwordsynsets(pos_data):
    sentiment = 0
    tokens_count = 0
    for word, pos in pos_data:
        if not pos:
            continue
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        if not lemma:
            continue
        
        synsets = wordnet.synsets(lemma, pos=pos)
        if not synsets:
            continue
    return synsets

def sentiwordnetanalysis(synsets):
    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())
    sentiment += swn_synset.pos_score() - swn_synset.neg_score()
    tokens_count += 1
    if not tokens_count:
        return 0
    if sentiment>0:
       return "Positive"
    if sentiment==0:
       return "Neutral"
    else:
       return "Negative"

In [138]:
print('texto:')
print(tweets['lemma_text'][0])
print('------------------------------------')
print('Polaridad según SentiWordNet')
print(swn.senti_synset(sentiwordsynsets(tweets['pos_tagged'][0])[0].name()))

texto:
  nancyleegrahn feel climate change question night exactly
------------------------------------
Polaridad según SentiWordNet
<precisely.r.01: PosScore=0.125 NegScore=0.0>


In [25]:
tweets['sentiment_swn'] = tweets['pos_tagged'].apply(sentiwordnetanalysis)
tweets[['text','lemma_text','sentiment_swn']].head(10)

,text,lemma_text,sentiment_swn
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question n...,Positive
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best line seco...,Neutral
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice hold cleveland wow,Negative
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hour debate...,Neutral
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump deliver high rati...,Negative
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal ex...,Positive
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman like happy heard go moderator a...,Positive
7,Going on #MSNBC Live with @ThomasARoberts arou...,go msnbc live thomasaroberts pm et,Neutral
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlight lizzwinstead ben carson brain...,Positive
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate prove batsask bada...,Neutral


In [26]:
swn_counts= pd.DataFrame(tweets[['sentiment_swn']].value_counts()).reset_index().rename(columns={0:'count','sentiment_swn':'sentiment'})
swn_counts.head(10)

,sentiment,count
0,Positive,6029
1,Neutral,3889
2,Negative,3797
3,0,156


In [119]:
px.pie(data_frame=swn_counts,names='sentiment',values='count',title='Análisis de sentimiento con SentiWordNet',hole=.5,color_discrete_sequence=['steelblue','lightgray','lightcoral','black'])

In [43]:
tweets[['text','lemma_text','sentiment_swn']][tweets['sentiment_swn']==0]

,text,lemma_text,sentiment_swn
43,RT @brownblaze: PLEASE RT. #KKKorGOP #GOPDebat...,brownblaze kkkorgop,0
143,RT @peddoc63: Go Carly📢\nGo Carly📢\nGo Carly📢\...,peddoc carly📢 carly📢 carly📢 carlyfiorina car...,0
253,RT @JosephKapsch: And then hillaryclinton @bar...,josephkapsch hillaryclinton barackobama joeb...,0
385,At least he was honest about it. #GOPDebate h...,honest,0
411,RT @Squintz1983: #GOPDebate pretty much http:/...,squintz pretty,0
...,...,...,...
13056,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0
13057,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0
13069,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0
13085,Huckabee. #GOPDebates http://t.co/Zy12bdUkx9,huckabee,0


Casi el 44% de los comentarios tienen una connotación positiva, segun SentiWordNet, mientras que los sentimientos neutrales o negativos se reparten de manera equitativa en un 28% aproximadamente. SentiWordNet no logró clasificar correctamente el 1.12% de los comentarios. Revisando algunos de los comentarios no clasificados, se observa que después de la limpieza y lematización, la mayoría coincide en que son pocas palabras no legibles, como nombres de usuarios o hashtagas, o palabras cortas.

### 5.2 Con librería TextBlob

La salida de TextBlob de polaridad es un flotante dentro del rango $[-1 , 1 ]$ dónde -1.0 es una polaridad negativa y 1.0 es positivo. Esta puntuación también puede ser igual a 0, que representa una evaluación neutral de una declaración, ya que no contiene ninguna palabra del conjunto de entrenamiento.

La librería se ha construido pensando, entre otras cosas, en el análisis particular de redes sociales.

In [28]:
#función para calcular polaridad con textblob
def textblob_polarity(text):
    return TextBlob(text).sentiment.polarity

#función para calcular sentimiento con textblob
def textblob_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [136]:
print('texto:')
print(tweets['lemma_text'][0])
print('------------------------------------')
print('Polaridad según TextBlob')
TextBlob(tweets['lemma_text'][0]).sentiment.polarity

texto:
  nancyleegrahn feel climate change question night exactly
------------------------------------
Polaridad según TextBlob


0.25

In [29]:
tweets['polarity_tb'] = tweets['lemma_text'].apply(textblob_polarity) 
tweets['sentiment_tb'] = tweets['polarity_tb'].apply(textblob_sentiment)
tweets[['text','lemma_text','sentiment_swn','sentiment_tb']].head(10)

,text,lemma_text,sentiment_swn,sentiment_tb
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question n...,Positive,Positive
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best line seco...,Neutral,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice hold cleveland wow,Negative,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hour debate...,Neutral,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump deliver high rati...,Negative,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal ex...,Positive,Negative
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman like happy heard go moderator a...,Positive,Positive
7,Going on #MSNBC Live with @ThomasARoberts arou...,go msnbc live thomasaroberts pm et,Neutral,Positive
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlight lizzwinstead ben carson brain...,Positive,Neutral
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate prove batsask bada...,Neutral,Neutral


In [30]:
tb_counts= pd.DataFrame(tweets[['sentiment_tb']].value_counts()).reset_index().rename(columns={0:'count','sentiment_tb':'sentiment'})
tb_counts.head(10)

,sentiment,count
0,Neutral,6949
1,Positive,4652
2,Negative,2270


In [115]:
px.pie(data_frame=tb_counts,names='sentiment',values='count',title='Análisis de sentimiento con TextBlob',hole=.5,color_discrete_sequence=['lightgray','steelblue','lightcoral'])

In [32]:
tweets[['text','lemma_text','sentiment_swn','sentiment_tb']][tweets['sentiment_swn']==0]

,text,lemma_text,sentiment_swn,sentiment_tb
43,RT @brownblaze: PLEASE RT. #KKKorGOP #GOPDebat...,brownblaze kkkorgop,0,Neutral
143,RT @peddoc63: Go Carly📢\nGo Carly📢\nGo Carly📢\...,peddoc carly📢 carly📢 carly📢 carlyfiorina car...,0,Neutral
253,RT @JosephKapsch: And then hillaryclinton @bar...,josephkapsch hillaryclinton barackobama joeb...,0,Neutral
385,At least he was honest about it. #GOPDebate h...,honest,0,Positive
411,RT @Squintz1983: #GOPDebate pretty much http:/...,squintz pretty,0,Positive
...,...,...,...,...
13056,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0,Neutral
13057,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0,Neutral
13069,RT @goldietaylor: Huckabee. #GOPDebates http:/...,goldietaylor huckabee,0,Neutral
13085,Huckabee. #GOPDebates http://t.co/Zy12bdUkx9,huckabee,0,Neutral


En el caso anterior, se observa que la cantidad de tweets con sentimiento positivo disminuye drásticamente, obteniendo apenas un 34% de los comentarios totales, mientras que los comentarios neutrales ascienden a la mitad. También se puede notar que TextBlob no tuvo problema en clasificar los tweets confusos o con pocas palabras, a diferencia de SentiWordNet.

### 5.3 Con librería Vader

VADER (Valence Aware Dictionary and sEntiment Reasoner) es una herramienta de análisis de sentimientos basada en reglas y léxico que se adapta específicamente a los sentimientos expresados ​​en las redes sociales.


El puntaje compuesto es una métrica que calcula la suma de todas las calificaciones de léxico que se han normalizado entre -1(más extremo negativo) y +1(más extremo positivo).

De acuerdo con https://es.acervolima.com/python-analisis-de-sentimiento-con-vader/#:~:text=VADER, se tomará como umbral el siguiente:

-sentimiento positivo:(puntaje compuesto> = 0.05)

-sentimiento negativo:(puntaje compuesto <= -0.05)

-sentimiento neutro: (puntaje compuesto> -0.05) y (puntaje compuesto <0.05)

In [33]:
vader_analyzer = SentimentIntensityAnalyzer()

In [109]:
print('texto:')
print(tweets['lemma_text'][0])
print('------------------------------------')
print('Polaridad según Vader')
vader_analyzer.polarity_scores(tweets['lemma_text'][0])

texto:
  nancyleegrahn feel climate change question night exactly
------------------------------------
Polaridad según Vader


{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [101]:
#función para calcular polaridad con vader
def vader_polarity(text):
    vs = vader_analyzer.polarity_scores(text)
    return vs['compound']

# función para calcular sentimiento con vader
def vader_sentiment(compound):
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05 :
        return 'Negative'
    else:
        return 'Neutral'

In [102]:
tweets['polarity_vdr'] = tweets['lemma_text'].apply(vader_polarity)
tweets['sentiment_vdr'] = tweets['polarity_vdr'].apply(vader_sentiment)
tweets[['text','lemma_text','sentiment_swn','sentiment_tb','sentiment_vdr']].head(10)

,text,lemma_text,sentiment_swn,sentiment_tb,sentiment_vdr
0,RT @NancyLeeGrahn: How did everyone feel about...,nancyleegrahn feel climate change question n...,Positive,Positive,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,scottwalker catch night scott best line seco...,Neutral,Positive,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,tjmshow mention tamir rice hold cleveland wow,Negative,Positive,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,robgeorge carly fiorina trending hour debate...,Neutral,Positive,Neutral
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,danscavino realdonaldtrump deliver high rati...,Negative,Positive,Neutral
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",gregabbott tx tedcruz day rescind illegal ex...,Positive,Negative,Negative
6,RT @warriorwoman91: I liked her and was happy ...,warriorwoman like happy heard go moderator a...,Positive,Positive,Positive
7,Going on #MSNBC Live with @ThomasARoberts arou...,go msnbc live thomasaroberts pm et,Neutral,Positive,Neutral
8,Deer in the headlights RT @lizzwinstead: Ben C...,deer headlight lizzwinstead ben carson brain...,Positive,Neutral,Neutral
9,RT @NancyOsborne180: Last night's debate prove...,nancyosborne night debate prove batsask bada...,Neutral,Neutral,Neutral


In [103]:
vdr_counts= pd.DataFrame(tweets[['sentiment_vdr']].value_counts()).reset_index().rename(columns={0:'count','sentiment_vdr':'sentiment'})
vdr_counts.head(10)

,sentiment,count
0,Positive,5840
1,Neutral,4320
2,Negative,3711


In [114]:
px.pie(data_frame=vdr_counts,names='sentiment',values='count',title='Análisis de sentimiento con Vader',hole=.5,color_discrete_sequence=['steelblue','lightgray','lightcoral'])

En el último caso, con el algoritmo de Vader, se comporta relativamente similar con el umbral seleccionado al algoritmo de SentiWordNet, con la diferencia de que Vader logró etiquetar correctamente la totalidad de los tweets, al igual que TextBlob. Esto sucede porque el léxico de estos últimos está adaptado para etiquetas de comentarios de redes sociales.

___

# 6. Visualización de Comparativas

In [144]:
swn_counts['algorithm']='SentiWordNet'
tb_counts['algorithm']='TextBlob'
vdr_counts['algorithm']='Vader'

In [146]:
sent_count=pd.concat([swn_counts,tb_counts,vdr_counts])
sent_count

,sentiment,count,module,algorithm
0,Positive,6029,SentiWordNet,SentiWordNet
1,Neutral,3889,SentiWordNet,SentiWordNet
2,Negative,3797,SentiWordNet,SentiWordNet
3,0,156,SentiWordNet,SentiWordNet
0,Neutral,6949,TextBlob,TextBlob
1,Positive,4652,TextBlob,TextBlob
2,Negative,2270,TextBlob,TextBlob
0,Positive,5840,NaN,Vader
1,Neutral,4320,NaN,Vader
2,Negative,3711,NaN,Vader


In [147]:
px.bar(sent_count,y='count',x='sentiment',color='algorithm',barmode='group')

Se observa un desbalance significativo entre TextBlob y los otros dos algoritmos probados. Es importante destacar que el hecho de que SentiWordNet haya arrojado valores nulos en el sentimiento de algunos comentarios, indica que estos pueden descartarse por la poca cantidad que representan del total (1.12%), no obstante, los valores nulos también hacen ver que aún hay área de oportunidad en el pre-procesamiento del texto y se podría realizar más limpieza de palabras, por ejemplo, agregando stopwords. 

Para futuros análisis, se usará el sentimiento de VADER como el definitivo, puesto que existe más similitud con el de SentiWordNet y está mejor adaptado a redes sociales, además éste último contiene valores nulos.

In [152]:
tweets.columns

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone', 'clean_text',
       'pos_tagged', 'lemma_text', 'sentiment_swn', 'polarity_tb',
       'sentiment_tb', 'polarity_vdr', 'sentiment_vdr'],
      dtype='object')

In [153]:
set(tweets['candidate'])

{'Ben Carson',
 'Chris Christie',
 'Donald Trump',
 'Jeb Bush',
 'John Kasich',
 'Marco Rubio',
 'Mike Huckabee',
 'No candidate mentioned',
 'Rand Paul',
 'Scott Walker',
 'Ted Cruz',
 nan}

In [168]:
candidates=tweets.groupby(['candidate','sentiment_vdr']).agg({'sentiment_vdr':['count']}).reset_index()
candidates.columns=['candidate','sentiment_vdr','count']
candidates.head(10)

,candidate,sentiment_vdr,count
0,Ben Carson,Negative,97
1,Ben Carson,Neutral,96
2,Ben Carson,Positive,211
3,Chris Christie,Negative,102
4,Chris Christie,Neutral,49
5,Chris Christie,Positive,142
6,Donald Trump,Negative,868
7,Donald Trump,Neutral,763
8,Donald Trump,Positive,1182
9,Jeb Bush,Negative,114


In [172]:
px.bar(candidates,y='count',x='sentiment_vdr',color='candidate',barmode='group')

Dejando fuera los tweets que no mencionaron a ningún candidato, el candidato más popular por mucho fue Donald Trump, con 1182 tweets positivos hacia él, 763 neutrales y 868 negativos.

___

# 6. Conclusión